# Multivariate Automatic Differentiation
The aim of this notebook is to provide understanding of the basic princples of automatic differentiation with trackers and operator overloading.

## Computationnal Graph
Operations to manipulate the directed acyclic computationnal graphs created using Julia's built-in dictionnary data structure.

In [1]:

function AddEdge(Edges, source, sink)
    if source in keys(Edges) merge!(Edges, IdDict(source=>Set([sink, get(Edges, source, false)...])))
    else merge!(Edges, IdDict(source=>Set([sink]))) end 
end

function RemoveEdge(Edges, source, sink)
    if source in keys(Edges) delete!(get(Edges, source, false), sink) end
end

function ReverseEdges(Edges)::IdDict
    ReversedEdges = IdDict{}
    for (source, sinks) in edges
        for sink in sinks AddEdge(ReversedEdges, source, sink) end
    end
    return ReversedEdges
end

function HasIncomingEdge(Edges, Node)::Bool
    for sinks in values(Edges)  if (Node in sinks) return true end end
    return false
end


function KahnTopoSort(Nodes::Set, edges::IdDict)
    # Kahn's topological sorting algorithm
    # edges is a DAG in the form of a dictionnary (source => sinks)
    Edges = deepcopy(edges)
    Sorted = []
    NoIncomingEdges = Set( [Node for Node in Nodes  if !HasIncomingEdge(Edges, Node)] )
    while !isempty(NoIncomingEdges)
        source = pop!(NoIncomingEdges)
        Sorted = cat(Sorted, [source], dims=1)
        sinks = get(Edges, source, false)
        if sinks!=false for sink in sinks
            RemoveEdge(Edges, source, sink)
            if !HasIncomingEdge(Edges, sink) NoIncomingEdges = union(NoIncomingEdges, Set(sink)) end
        end end
    end
    return Sorted
end


Nodes = Set([1, 2, 3, 4, 5])
Edges = IdDict(1=>Set([2, 3]), 4=>Set([5]), 2=>Set([4, 5]), 3=>Set([4]))
@show KahnTopoSort(Nodes, Edges) #should return [5, 4, 2, 3, 1] or [5, 2, 4, 3, 1]

KahnTopoSort(Nodes, Edges) = Any[1, 2, 3, 4, 5]


5-element Vector{Any}:
 1
 2
 3
 4
 5

## Forward Pass
Almost every function can be broken down into a chain of very simple operations.

In [2]:
import Base.:+
import Base.:-
import Base.:*
import Base.:/
import Base.:^
import Base.sin
import Base.cos

function AddJacobian(Jacobians, source, sink, Jacobian)
    # adds jacobian of sink with respect to source to jacobians
    merge!(Jacobians, IdDict((source, sink) => Jacobian)) 
end

function GenNode(Nodes)
    # generate a new Node (Nodeentification in the graph)
    Node = convert(Int64, floor(10000000 * rand()))
    while Node in Nodes
        Node = convert(Int64, floor(10000000 * rand()))
    end
    Nodes = union!(Nodes, Node)
    return Node
end

mutable struct Tracked{T} <: Real 
    val::T
    Node::Int64 # identification in the computationnal graph
    Tracked(val, Nodes) = return new{typeof(val)}(val, GenNode(Nodes))
    Tracked(val, Node) = return new{typeof(val)}(val, Node) 
end

#Base.convert(::Type{AbstractFloat}, t::Type{Tracked{Float64}}) = t
#Base.promote_rule(::Type{AbstractFloat}, ::Type{Tracked{Float64}}) =AbstractFloat

for t in (Symbol(Integer), Symbol(AbstractFloat))
    eval(
    quote
    GetJacobian(f::typeof(+), a::($t), b::Tracked) = 1
    GetJacobian(f::typeof(+), a::Tracked, b::($t)) = 1

    GetJacobian(f::typeof(-), a::($t), b::Tracked) = -1
    GetJacobian(f::typeof(-), a::Tracked, b::($t)) = 1

    GetJacobian(f::typeof(*), a::($t), b::Tracked) = a
    GetJacobian(f::typeof(*), a::Tracked, b::($t)) = b

    GetJacobian(f::typeof(/), a::($t), b::Tracked) = (-1)/b.val^2
    GetJacobian(f::typeof(/), a::Tracked, b::($t)) = 1/b

    GetJacobian(f::typeof(^), a::Tracked, b::($t)) = b*a.val^(b-1)
    GetJacobian(f::typeof(^), a::($t), b::Tracked) = a^(b.val)*log(a)
    
    GetJacobian(f::typeof(sin), a::Tracked) = cos(a.val)
    GetJacobian(f::typeof(cos), a::Tracked) = -sin(a.val)
    end
    )
end


function WithGradient(f, x, w::Set)
    
    # Overcharge the operators to create a computationnal graph as well as 
    # the intermediate jacobians for backpropagation at a later stage

    global Nodes = deepcopy(w)
    global Edges = IdDict()
    global Jacobians = IdDict()

    for op in (Symbol(+), Symbol(-), Symbol(*), Symbol(/), Symbol(^))
        for t in (Symbol(Integer), Symbol(AbstractFloat))

        eval(:(global function ($op)(a::T, b::Tracked) where {T <: ($t)}
        Node = GenNode(Nodes)
        J = GetJacobian(($op), a, b)
        AddEdge(Edges, b.Node, Node)
        AddJacobian(Jacobians, b.Node, Node, J)
        return Tracked(($op)(a, b.val), Node) end))

        eval(:(global function ($op)(a::Tracked, b::T) where {T <: ($t)}
        Node = GenNode(Nodes)
        J = GetJacobian(($op), a, b)
        AddEdge(Edges, a.Node, Node)
        AddJacobian(Jacobians, a.Node, Node, J)
        return Tracked(($op)(a.val, b), Node) end))

        eval(:(global function ($op)(a::Tracked, b::Tracked)
        Node = GenNode(Nodes)
        Ja = GetJacobian(($op), a, b.val)
        AddEdge(Edges, a.Node, Node)
        AddJacobian(Jacobians, a.Node, Node, Ja)
        Jb = GetJacobian(($op), a.val, b)
        AddEdge(Edges, b.Node, Node)
        AddJacobian(Jacobians, b.Node, Node, Jb)
        return Tracked(($op)(a.val, b.val), Node) end))

        end

    end

    for op in (Symbol(sin), Symbol(cos))
        eval(
            :(
            global function ($op)(a::Tracked)
                Node = GenNode(Nodes)
                J = GetJacobian(($op), a)
                AddEdge(Edges, a.Node, Node)
                AddJacobian(Jacobians, a.Node, Node, J)
                return Tracked(($op)(a.val), Node) 
            end
            )
        )
    end

    y = Base.invokelatest(f, x)
    return (y, Nodes, Edges, Jacobians)
end


WithGradient (generic function with 1 method)

## Backward Pass
$$\frac{d f \circ g}{dx} = \frac{d f}{d g} \frac{dg}{dx}$$

In [3]:

function Backprop(f, x, w)::IdDict
    y, Nodes, Edges, Jacobians = WithGradient(f, x, w) # forward pass (intermediate jacobians are computed)
    TopoSortNodes = KahnTopoSort(Nodes, Edges) 
    ChainedJacobians = IdDict{Any, Float64}(y.Node=>1)
    for source in reverse(TopoSortNodes[1:end-1])
        CJ = 0
        sinks = get(Edges, source, false)
        for sink in sinks
            CJ += get(Jacobians, (source, sink), false) * get(ChainedJacobians, sink, false)
        end
        merge!(ChainedJacobians, IdDict(source=>CJ))
    end
    Gradients = IdDict([source=>get(ChainedJacobians, source, false) for source in keys(ChainedJacobians) if source in w])
    return Gradients
end

Backprop (generic function with 1 method)

In [4]:
oepli(w, x) = 2 * (((2*w + x) - w) * w) + sin(w^2)
ID = 12345
w = Tracked(7.0, ID)
grads = Backprop(x -> oepli(w, x), 12.0, Set([ID])) # should be approx. 56.21
println(grads)

jutyp(w) = sin(w)^2
ID = 12345
w = Tracked(25.0, ID)
grads = Backprop(_ -> jutyp(w), nothing, Set([ID])) # should be approx. -0.26
println(grads)

tyamelki(w1, w2) = (cos(w2)^3 * 4sin(w1))^2 + cos(w1*w2)^2
w1 = Tracked(13.2, 420); w2 = Tracked(89, 007)
grads = Backprop(_ -> tyamelki(w1, w2), nothing, Set([420, 007])) # should be approx. (27.53, 3.0428)
println(grads)


IdDict(12345 => 56.20829561241092)
IdDict(12345 => -0.26237485370392877)


IdDict(420 => 27.53002464314667, 7 => 3.0428042273213602)
